In [1]:
from conf import *

In [2]:
def load_jsons_as_cols(df, JSON_COLUMNS):
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

    return df

In [3]:
# trasforma liste di dizionari in un solo dizionario
def unpack(col):
    mod = []
    for row in col:
        try:
            mod += eval(row)
        except:
            mod += row

    return pd.Series(mod)


def correction_nan(x):
    if type(x) == type(0.0):
        return {}
    else:
        return x


def mappa(fun, ite):
    return [fun(x) for x in ite]


# è una sola la colonna che ricevo e la trasformo in un dict
def from_dict_to_frame(df, col):
    # return pd.DataFrame(df[col])

    df[col] = mappa(correction_nan, df[col])

    new = []
    for c in df[col]:
        new += [c]
    new2 = pd.DataFrame.from_records(new)
    # corretto new2 -> print(type(new2))

    return new2

In [4]:
# trasforma n colonne con un dict all interno in un dizionario che le comprende
def unfold(df, cols,target):
    for col in cols:
        if col not in target:
            df = df.merge(from_dict_to_frame(df, col), right_index=True, left_index=True)
            df = df.drop(col, axis=1)

    return df

In [5]:
def find_lists(df,target):
    lists_to_dump = []
    for x in range(len(df.columns)):
        try:
            if (type(eval(df.iloc[0, x])) == type([])) & (df.columns[x] not in target):
                lists_to_dump += [df.columns[x]]
        except:
            try:
                if (type(df.iloc[0, x]) == type([])) & (df.columns[x] not in target):
                    lists_to_dump += [df.columns[x]]
            except:
                pass

    return lists_to_dump

In [6]:
def find_dicts(df,target):
    dicts_to_dump = []
    for x in range(len(df.columns)):
        try:
            if (type(df.iloc[0, x]) == type({})) & (df.columns[x] not in target):
                dicts_to_dump += [df.columns[x]]
        except:
            try:
                if (type(eval(df.iloc[0, x])) == type({})) & (df.columns[x] not in target):
                    dicts_to_dump += [df.columns[x]]
            except:
                pass
    if "transactionRevenue" in dicts_to_dump:
        print("errore in find_dicts")
    return dicts_to_dump

In [7]:
def drop_const(df,target):
    for col in df.columns:
        if (df[col].astype(str).nunique(dropna=False) == 1) & (col not in target):
            df = df.drop(col, axis=1)

    return df

In [8]:
def load_df(csv_path,n_rows,target):
    JSON_COLUMNS = ["device", "geoNetwork", "totals", "trafficSource"]

    df = pd.read_csv(csv_path,
                     converters={column: json.loads for column in JSON_COLUMNS}
                     ,nrows = n_rows, dtype={"fullVisitorId": "str"}
                     )
    
    df = load_jsons_as_cols(df,JSON_COLUMNS)
    
    #while ((len(find_dicts(df,target)) > 0) | (len(find_lists(df,target)) > 0)):

    if len(find_lists(df,target)) > 0:
        for col in find_lists(df,target):
            df[col] = unpack(df[col])
            
    while len(find_dicts(df,target)) > 0:
        df = unfold(df, find_dicts(df,target),target)
        
    '''summa=0
    new_col = []
    #ciclo le colonne che ho scoperto contenere liste
    for col in find_lists(df,target):
        #ciclo le righe che contenti liste
        for row in df[col]:
            #ciclo la lista contenete dizionari
            if len(row)>0:
                #print(row[:3],len(row))
                for d in row:
                    try:
                        summa += float(d['productPrice'])
                    except:
                        pass
                new_col += [summa]
                summa = 0
    df["transactionRevenue"] = pd.Series(new_col)'''
    
    return df

In [9]:
target = ["fullVisitorId","transactionRevenue"]
train_df = load_df(train_file,10000,target)

In [23]:
train_df.head()

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,index,value,customDimensions,customMetrics,customVariables,dataSource,eventInfo,experiment,hitNumber,hour,isEntrance,isExit,isInteraction,latencyTracking,minute,product,promotion,promotionActionInfo,publisher_infos,referer,time,type,exitScreenName,landingScreenName,screenDepth,screenName,contentGroup1,contentGroup2,contentGroup3,contentGroup4,contentGroup5,contentGroupUniqueViews1,contentGroupUniqueViews2,contentGroupUniqueViews3,previousContentGroup1,previousContentGroup2,previousContentGroup3,previousContentGroup4,previousContentGroup5,action_type,option,step,isFatal,currencyCode_x,transactionId_x,hostname,pagePath,pagePathLevel1,pagePathLevel2,pagePathLevel3,pagePathLevel4,pageTitle,searchCategory,searchKeyword,hasSocialSourceReferral,socialInteractionNetworkAction,socialNetwork,affiliation,currencyCode_y,localTransactionRevenue,localTransactionShipping,localTransactionTax,transactionId_y,transactionRevenue,transactionShipping,transactionTax
0,Organic Search,20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,Western Europe,1,1,1,1,1,NaN,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,water bottle,organic,NaN,google,4,EMEA,[],[],[],web,NaN,[],1,17,True,True,True,NaN,0,"[{'productSKU': 'GGOEGDHC074099', 'v2ProductNa...",[],NaN,[],https://www.google.co.uk/search?q=water+bottle...,0,PAGE,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/google+redesig...,0,shop.googlemerchandisestore.com/google+redesig...,(not set),Bags,(not set),(not set),(not set),NaN,1,NaN,(entrance),(entrance),(entrance),(entrance),(entrance),0,NaN,1,True,USD,NaN,shop.googlemerchandisestore.com,/google+redesign/bags/water+bottles+and+tumblers,/google+redesign/,/bags/,/water+bottles+and+tumblers,,Water Bottles & Tumblers | Drinkware | Google ...,NaN,NaN,No,:,(not set),NaN,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Referral,20171016,8934116514970143966,Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,

In [11]:
#train_df["product"][0]

In [12]:
#train_df.dtypes

In [13]:
type(train_df.dtypes)

pandas.core.series.Series

In [14]:
train_df.dtypes

channelGrouping                                      object
date                                                  int64
fullVisitorId                                        object
socialEngagementType                                 object
visitId                                               int64
visitNumber                                           int64
visitStartTime                                        int64
device.browser                                       object
device.browserSize                                   object
device.browserVersion                                object
device.deviceCategory                                object
device.flashVersion                                  object
device.isMobile                                        bool
device.language                                      object
device.mobileDeviceBranding                          object
device.mobileDeviceInfo                              object
device.mobileDeviceMarketingName        

In [15]:
train_df.dtypes[1]

dtype('int64')

In [16]:
type(train_df.dtypes[1])

numpy.dtype

In [22]:
train_df["transactionRevenue"].fillna(0.0).astype(float).sum()

1786420000.0